In [1]:
import edhec_risk_kit as erk
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthBegin
import plotly.graph_objs as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
from dash_extensions.callback import CallbackGrouper
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from dash.dependencies import Input, Output, State
import dash_table as dt
%load_ext autoreload
%autoreload 2



START LAB 201

In [3]:
brka_d = pd.read_csv('data/brka_d_ret.csv', parse_dates=True, index_col=0)
brka_m = brka_d.resample('1M').apply(erk.cumulate)
brka_m.index -= MonthBegin(1)
fff = erk.get_df(filename='data/F-F_Research_Data_Factors_m.csv', start_period=None, end_period=None,
                 format='%Y%m', reqd_strategies=None)
rr_capm = erk.regress(brka_m, fff[['Mkt-RF', 'RF']], '1990', rfcol='RF')
rr_ffm = erk.regress(brka_m, fff, '1990', rfcol='RF')
rr_ffm_nint = erk.regress(brka_m, fff, '1990', rfcol='RF', intercept=False)
print(rr_ffm.summary())
print(rr_ffm_nint.summary())

                            OLS Regression Results                            
Dep. Variable:                   BRKA   R-squared:                       0.317
Model:                            OLS   Adj. R-squared:                  0.311
Method:                 Least Squares   F-statistic:                     53.29
Date:                Fri, 22 May 2020   Prob (F-statistic):           2.59e-28
Time:                        09:20:09   Log-Likelihood:                 567.01
No. Observations:                 348   AIC:                            -1126.
Df Residuals:                     344   BIC:                            -1111.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.7096      0.063     11.350      0.0

START LAB 202

In [4]:
ind = erk.get_df('data/ind30_m_vw_rets.csv', start_period='2000', format='%Y%m', mode='return', end_period=None, reqd_strategies=None)
mgr_r = 0.3*ind["Beer"] + .5*ind["Smoke"] + 0.2*np.random.normal(scale=0.15/(12**.5), size=ind.shape[0])
weights = erk.style_analyze(mgr_r, ind, start_period='2000')
style_coeff = erk.regress(mgr_r, ind, intercept=False, excess_mkt=False).params


PLOT FUNCTION

In [5]:
def plot_style(wts_coeff: pd.Series):
    wts_coeff.sort_values(ascending=False, inplace=True)
    wt_data = [go.Bar(x=wts_coeff.index, y=wts_coeff)]
    pyo.iplot(wt_data)

STYLE MATCHING PLOT USING QUADRATIC OPTIMIZER - MORE ACCURATE

In [6]:
plot_style(weights)

STYLE MATCHING PLOT USING REGRESSION COEFF

In [7]:
plot_style(style_coeff)

STYLE ANALYZE OF BRKA_M FOR LAST DECADE FROM 2009

In [8]:
wts_09 = erk.style_analyze(brka_m, ind, start_period='2009')
plot_style(wts_09)



PENDING WORK - DOWNLOAD ANY MUTUAL FUND HISTORICAL DATA FROM YF AND ANALYZE USING FFF




LAB 203 START




In [9]:
# TO_PERIOD ('m') IS THEN CONVERTED TO TIMESTAMP OBJECT WHILE PLOTTING
ind_ew = erk.get_df('data/ind30_m_ew_rets.csv', to_per=True)
ind_vw = erk.get_df('data/ind30_m_vw_rets.csv', to_per=True)
sharpe_ratio_df = pd.DataFrame({'VW': erk.risk_info(ind_vw['1945':], only_sharpe=True),
                               'EW': erk.risk_info(ind_ew['1945':], only_sharpe=True)})


PLOT VW AND EW SHARPE RATIO




In [10]:
sr_data = [go.Bar(x=sharpe_ratio_df.index,
                  y=sharpe_ratio_df[col]) for col in sharpe_ratio_df.columns]
pyo.iplot(sr_data)

% OF CASES WHERE SR(EW) > SR(VW)




In [11]:
(sharpe_ratio_df['EW'] > sharpe_ratio_df['VW']).sum()/sharpe_ratio_df.shape[0]

0.6333333333333333

ROLLING WINDOW SR CALC FOR VW AND EW MARKET PF




In [12]:
sharpe_ratio_rolling_df = pd.DataFrame({
    'VW': ind_vw.rolling(window='1825D').apply(erk.risk_info, kwargs={'only_sharpe': True, 'rf': 0.03}, raw=True).mean(axis=1)['1945':].dropna(),
    'EW': ind_ew.rolling(window='1825D').apply(erk.risk_info, kwargs={'only_sharpe': True, 'rf': 0.03}, raw=True).mean(axis=1)['1945':].dropna()
})


PLOT ROLLING SR




In [13]:
sr_r_data = [go.Scatter(x=sharpe_ratio_rolling_df.index.to_timestamp(),
                      y=sharpe_ratio_rolling_df[col],
                      name=col) for col in sharpe_ratio_df.columns]
pyo.iplot(sr_r_data)

LAB START 204
CREATE CUSTOM ROLLING WINDOW TO HANDLE MULTI DIMENSION DATAFRAME FOR COMPUTATION
ULT - TRYING TO GET ROLLING RETURNS ACCORDING TO SPECIFIED WEIGHTING SCHEMES


In [2]:
ind_vw_74 = erk.get_df('data/ind49_m_vw_rets.csv', to_per=True, start_period='1974')
ind_ew_74 = erk.get_df('data/ind49_m_ew_rets.csv', to_per=True, start_period='1974')
ind_mkt_wts_74 = erk.plot_corr_mktret(ind_ret_filename='data/ind49_m_vw_rets.csv',
                                       n_firms_filename='data/ind49_m_nfirms.csv',
                                       size_filename='data/ind49_m_size.csv',
                                       start_period='1974',
                                       end_period=None,
                                       to_per=True,
                                       retrieve_mkt_cap_wts=True,
                                       format='%Y%m')

In [3]:
mw_pf = erk.bt_roll(ind_ew_74, weighting_scheme=erk.weight_cw, window=36, cap_wts=ind_mkt_wts_74)
ew_pf = erk.bt_roll(ind_ew_74, weighting_scheme=erk.weight_ew, window=36, cap_wts=ind_mkt_wts_74)
ewt_pf = erk.bt_roll(ind_ew_74, weighting_scheme=erk.weight_ew, window=36, cap_wts=ind_mkt_wts_74, max_cw_mult=5, microcap_threshold=0.005)
btr = {'EW': ew_pf,
       'MW': mw_pf,
       'EWT': ewt_pf}


PLOT INDEX AND DISPLAY RISK INFO


In [9]:
def plot_wealth_index(btr: dict):
    btr_df = pd.DataFrame(btr)
    btr_df.dropna(inplace=True)
    btr_wealth_index = pd.DataFrame({key+'_wealth_index': erk.drawdown(btr[key], retrive_index=True, is1p=False) for key in btr_df.columns})
    wealth_data=[go.Scatter(x=btr_wealth_index.index.to_timestamp(),
                     y=btr_wealth_index[str],
                     name=str) for str in btr_wealth_index.columns]
    pyo.iplot(wealth_data)
    return erk.risk_info(btr_df)

START QUIZ COURSE 2 WEEK 1


In [17]:
ind_vw_91 = erk.get_df('data/ind49_m_vw_rets.csv', to_per=True, start_period='1991')
fff_91 = erk.get_df(filename='data/F-F_Research_Data_Factors_m.csv', start_period='1991', end_period=None,
                 format='%Y%m', reqd_strategies=None, to_per=True)
fff_91_capm = fff_91[['Mkt-RF', 'RF']]
rr_capm_beer_1991 = erk.regress(ind_vw_91['Beer'], fff_91_capm, '1991', rfcol='RF')
rr_capm_beer_2013 = erk.regress(ind_vw_91['Beer'], fff_91_capm, '2013', rfcol='RF')
rr_capm_steel_1991 = erk.regress(ind_vw_91['Steel'], fff_91_capm, '1991', rfcol='RF')
rr_capm_steel_2013 = erk.regress(ind_vw_91['Steel'], fff_91_capm, '2013', rfcol='RF')

In [18]:
print(rr_capm_steel_2013.summary())

                            OLS Regression Results                            
Dep. Variable:                  Steel   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.463
Method:                 Least Squares   F-statistic:                     62.18
Date:                Tue, 19 May 2020   Prob (F-statistic):           2.97e-11
Time:                        18:43:00   Log-Likelihood:                 116.15
No. Observations:                  72   AIC:                            -228.3
Df Residuals:                      70   BIC:                            -223.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         1.4169      0.180      7.885      0.0

In [19]:
regress_results = erk.regress(ind_vw_91, fff_91_capm, '1991', '1993', rfcol='RF').params
regress_results.columns = ind_vw_91.columns
regress_results.loc['Mkt-RF'].sort_values(ascending=False).tail()

Oil      0.550604
Coal     0.529958
Util     0.379527
Mines    0.274762
Gold    -0.553289
Name: Mkt-RF, dtype: float64

In [20]:
regress_results_ff = erk.regress(ind_vw_91, fff_91, '1991', rfcol='RF').params
regress_results_ff.columns = ind_vw_91.columns
regress_results_ff.loc['SMB'].sort_values(ascending=False).tail()

Insur   -0.251649
Food    -0.267873
Smoke   -0.307794
Drugs   -0.319486
Beer    -0.358655
Name: SMB, dtype: float64

In [21]:
regress_results_ff = erk.regress(ind_vw_91, fff_91, '1991', rfcol='RF').params
regress_results_ff.columns = ind_vw_91.columns
regress_results_ff.loc['HML'].sort_values(ascending=False).tail()

Drugs   -0.228132
LabEq   -0.261102
Chips   -0.600139
Hardw   -0.690417
Softw   -0.852422
Name: HML, dtype: float64

START WEEK 2 - FOCUS ON COV ESTIMATORS
USING GMV PF:
    1. SAMPLE COV MATRIX USING COV FN
    2. CONST CORR MATRIX
    3. STAT SHRINKAGE BET SAMPLE AND EQ DELTA 0.5


In [3]:
gmv_sample_cov = erk.bt_roll(ind_ew_74, weighting_scheme=erk.weight_gmv, window=36)
gmv_const_cov = erk.bt_roll(ind_ew_74, weighting_scheme=erk.weight_gmv, window=36, cov_estimator=erk.cc_cov)
gmv_stat_cov = erk.bt_roll(ind_ew_74, weighting_scheme=erk.weight_gmv, window=36, cov_estimator=erk.stat_shrinkage_cov, delta=0.5)

In [4]:
btr={}
btr.update(GMV_Sample_Cov=gmv_sample_cov,
           GMV_Const_Cov=gmv_const_cov,
           GMV_Stat_Cov=gmv_stat_cov)


PLOT WEALTH INDEX AND RISK INFO DETAILS

In [7]:
plot_wealth_index(btr)



,GMV_Const_Cov,GMV_Stat_Cov,GMV_Sample_Cov
ann_ret,0.141794,0.147691,0.159488
ann_vol,0.102668,0.099442,0.097030
sharpe_ratio,1.088885,1.183516,1.334516
semi_dev,0.021709,0.021724,0.021077
Kurtosis,1.458269,1.718804,2.054815
Skew,-0.500369,-0.525175,-0.530828
VaR,-0.042145,-0.040089,-0.037836
CVaR,-0.061776,-0.060219,-0.057669
Drawdown,-0.421375,-0.409070,-0.332092


PLOT STACKED BAR CHART WRT WEIGHTS UNDER EACH WEIGHTING SCHEME




In [3]:
r = ind_ew_74['1974':]
cap_wts = ind_mkt_wts_74['1974':]
wts = {
    'EW': erk.weight_ew(r),
    'EWT': erk.weight_ew(r, cap_wts=cap_wts, max_cw_mult=5, microcap_threshold=0.005),
    'CW': erk.weight_cw(r, cap_wts=cap_wts),
    'GMV_S_COV': erk.weight_gmv(r, cov_estimator=erk.sample_cov),
    'GMV_C_COV': erk.weight_gmv(r, cov_estimator=erk.cc_cov),
    'GMV_SHRINK_0.5': erk.weight_gmv(r, cov_estimator=erk.stat_shrinkage_cov)
}
wts = pd.DataFrame(wts)

In [4]:
wts_data = [go.Bar(x=wts.columns,
                   y=wts.loc[sector],
                   name=sector) for sector in wts.index]
fig = go.Figure(data=wts_data,
                layout=dict(barmode='stack'))
pyo.iplot(fig)

TEST WEEK 2




In [2]:
ind_vw_97 = erk.get_df('data/ind30_m_vw_rets.csv', to_per=True, start_period='1997')
ind_mkt_wts_97 = erk.plot_corr_mktret(ind_ret_filename='data/ind30_m_vw_rets.csv',
                                       n_firms_filename='data/ind30_m_nfirms.csv',
                                       size_filename='data/ind30_m_size.csv',
                                       start_period='1997',
                                       end_period=None,
                                       to_per=True,
                                       retrieve_mkt_cap_wts=True,
                                       format='%Y%m')

In [4]:
mw_pf = erk.bt_roll(ind_vw_97, weighting_scheme=erk.weight_cw, window=36, cap_wts=ind_mkt_wts_97)
ew_pf = erk.bt_roll(ind_vw_97, weighting_scheme=erk.weight_ew, window=36, cap_wts=ind_mkt_wts_97)
btr1 = {'EW': ew_pf,
       'MW': mw_pf}

In [10]:
plot_wealth_index(btr1)



,MW,GMV_Sample_Cov,GMV_Stat_shrink,EWT,EW
ann_ret,0.064564,0.066307,0.068468,0.076838,0.077614
ann_vol,0.151274,0.117410,0.114571,0.140774,0.158685
sharpe_ratio,0.228488,0.309233,0.335752,0.332717,0.300054
semi_dev,0.032552,0.026149,0.025509,0.031184,0.034696
Kurtosis,1.248421,1.469165,1.264935,1.664374,2.317042
Skew,-0.552933,-0.741107,-0.731286,-0.649872,-0.538531
VaR,-0.073620,-0.057504,-0.055881,-0.067937,-0.075253
CVaR,-0.096591,-0.082707,-0.076098,-0.093766,-0.106220
Drawdown,-0.518857,-0.421287,-0.393404,-0.487413,-0.532980


In [5]:
ewt_pf = erk.bt_roll(ind_vw_97, weighting_scheme=erk.weight_ew, window=36, cap_wts=ind_mkt_wts_97, max_cw_mult=2, microcap_threshold=0.01)
btr1.update(EWT=ewt_pf)

In [6]:
te_ew_mw = erk.tracking_error(act_rets=ew_pf, exp_rets=mw_pf)
te_ewt_mw = erk.tracking_error(act_rets=ewt_pf, exp_rets=mw_pf)

In [7]:
gmv_sample_cov = erk.bt_roll(ind_vw_97, weighting_scheme=erk.weight_gmv, window=36)
btr1.update(GMV_Sample_Cov=gmv_sample_cov)

In [8]:
gmv_stat_cov = erk.bt_roll(ind_vw_97, weighting_scheme=erk.weight_gmv, window=36, cov_estimator=erk.stat_shrinkage_cov, delta=0.25)
btr1.update(GMV_Stat_shrink=gmv_stat_cov)

BLACK LITTERMAN MODEL




In [4]:
app = dash.Dash()
cg = CallbackGrouper()
checkoptions = [
    {'label': 'Weights', 'value': 0},
    {'label': 'Cov_Mat', 'value': 1}
]


def get_bl_results(wts_prior: pd.Series, sigma_prior: pd.DataFrame, delta, tau, p: pd.DataFrame = None,
                   q: pd.Series = None, omega=None, scale=True, wts_he=False):
    pi = erk.rev_opt_implied_returns(delta, sigma_prior, wts_prior).to_numpy()
    omega = omega.to_numpy()
    diag_el = np.diag(omega)
    if not diag_el.any():
        omega = None
    bl_mu, bl_sigma, omega = erk.black_litterman(wts_prior, sigma_prior, p, q, omega, delta, tau)
    prior_wts, posterior_wts, wts_diff = erk.get_optimal_wts(sigma_prior, bl_sigma, pi, bl_mu, delta, tau,
                                                             wts_he=wts_he, scale=scale)
    wts = [prior_wts, posterior_wts, wts_diff]
    return pi, bl_mu, bl_sigma, wts, omega


def percent_format(col: pd.Series):
    return col.map('{:.4%}'.format)


app.layout = html.Div(
    [html.Div([html.Div([dcc.Checklist(id='local_file', options=[{'label': 'Y', 'value': 1}], value=[],
                                       labelStyle={'display': 'inline-block'}),
                         dcc.Checklist(id='hard_coded', options=[{'label': 'H', 'value': 1}], value=[],
                                       labelStyle={'display': 'inline-block'})
                         ]),
               html.Label(children='Enter list of asset names: '),
               dcc.Input(id='asset_list', value='', type='text')], style={'display': 'inline-block'}),
     html.Div([dcc.Checklist(id='display_table', options=checkoptions, value=[],
                             labelStyle={'display': 'inline-block'})]),
     html.Div(id='dump_vw_df', style={'display': 'none'}),
     html.Div(id='dump_wt_df', style={'display': 'none'}),
     html.Div(id='dump_wt_hc_df', style={'display': 'none'}),
     html.Div(id='dump_sigma_hc_df', style={'display': 'none'}),
     html.Div(id='dump_no', style={'display': 'none'}),
     html.Button(id='proceed_to_views', children='PROCEED', n_clicks=0, style={'display': 'inline-block'}),
     html.Div(id='wts_table_container', children=[dt.DataTable(id='wts_table', editable=True)],
              style={'display': 'none'}),
     html.Div(id='rhos_table_container', children=[dt.DataTable(id='rhos_table', editable=True)],
              style={'display': 'none'}),
     html.Div(id='vol_table_container', children=[dt.DataTable(id='vol_table', editable=True)],
              style={'display': 'none'}),
     html.Div(id='cov_table_container', children=[dt.DataTable(id='cov_table', editable=True)],
              style={'display': 'none'}),
     html.Div(id='view_params', children=[html.Label(children='Enter no of views: '),
                                          dcc.Input(id='no_views', value='', type='number'),
                                          html.Label(children='Enter tau: '),
                                          dcc.Input(id='tau', value=0.02, type='number'),
                                          html.Label(children='Enter delta: '),
                                          dcc.Input(id='delta', value=2.5, type='number'),
                                          html.Button(id='update_views', children='UPDATE VIEWS', n_clicks=0)],
              style={'display': 'none'}),
     html.Div(id='view_table_container', children=[dt.DataTable(id='view_table', editable=True)],
              style={'display': 'none'}),
     html.Div(id='pick_table_container', children=[dt.DataTable(id='pick_table', editable=True)],
              style={'display': 'none'}),
     html.Div(id='omega_table_container', children=[dt.DataTable(id='omega_table', editable=True)],
              style={'display': 'none'}),
     html.Button(id='update', children='UPDATE', n_clicks=0, style={'display': 'none'}),
     html.Div(dt.DataTable(id='bl_results_table'),
              style={'display': 'flex', 'justify-content': 'space-evenly', 'padding-top': '25px'}),
     html.Div(html.Pre(id='disp_bl_cov', style={'display': 'block'}),
              style={'display': 'flex', 'justify-content': 'space-evenly', 'padding-top': '25px'}),
     html.Div(html.Pre(id='disp_omega', style={'display': 'block'}),
              style={'display': 'flex', 'justify-content': 'space-evenly', 'padding-top': '25px'})
     ])


@cg.callback([Output('asset_list', 'value'),
              Output('dump_no', 'children'),
              Output('dump_vw_df', 'children'),
              Output('dump_wt_df', 'children'),
              Output('display_table', 'options')],
             [Input('local_file', 'value')])
def upd_asset_list(localfile):
    if localfile:
        ind_vw_2014 = erk.get_df('data/ind49_m_vw_rets.csv', to_per=True, start_period='2013', reqd_strategies=['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food'])
        ind_mkt_wts_2014 = erk.plot_corr_mktret(ind_ret_filename='data/ind49_m_vw_rets.csv',
                                                n_firms_filename='data/ind49_m_nfirms.csv',
                                                size_filename='data/ind49_m_size.csv',
                                                start_period='2014',
                                                end_period=None,
                                                to_per=True,
                                                retrieve_mkt_cap_wts=True,
                                                format='%Y%m',
                                                reqd_strategies=['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food'])
        a = ind_vw_2014.columns
        asset_list = [' '.join(list(ind_vw_2014.columns))]
        json_vw = ind_vw_2014.to_json(date_format='iso', orient='table')
        json_wts = ind_mkt_wts_2014.to_json(date_format='iso', orient='table')
        checkoptions = []
        return asset_list, len(ind_vw_2014.columns), json_vw, json_wts, checkoptions


@cg.callback([Output('dump_wt_hc_df', 'children'),
              Output('dump_sigma_hc_df', 'children'),
              Output('dump_no', 'children'),
              Output('asset_list', 'value'),
              Output('display_table', 'options')],
             [Input('hard_coded', 'value')])
def hard_code_cov(hard_coded):
    if hard_coded:
        countries = ['AU', 'CA', 'FR', 'DE', 'JP', 'UK', 'US']
        asset_list = [' '.join(countries)]
        wts_prior = pd.Series([0.016, 0.022, 0.052, 0.055, 0.116, 0.124, 0.615], index=countries, name='wts_prior')
        rhos_prior = pd.DataFrame([[1.000, 0.488, 0.478, 0.515, 0.439, 0.512, 0.491],
                                   [0.488, 1.000, 0.664, 0.655, 0.310, 0.608, 0.779],
                                   [0.478, 0.664, 1.000, 0.861, 0.355, 0.783, 0.668],
                                   [0.515, 0.655, 0.861, 1.000, 0.354, 0.777, 0.653],
                                   [0.439, 0.310, 0.355, 0.354, 1.000, 0.405, 0.306],
                                   [0.512, 0.608, 0.783, 0.777, 0.405, 1.000, 0.652],
                                   [0.491, 0.779, 0.668, 0.653, 0.306, 0.652, 1.000]], index=countries,
                                  columns=countries)
        vol_prior = pd.DataFrame([0.160, 0.203, 0.248, 0.271, 0.210, 0.200, 0.187], index=countries, columns=['vol'])
        sigma_prior = vol_prior @ vol_prior.T * rhos_prior
        checkoptions = []
        return wts_prior.to_json(orient='table'), sigma_prior.to_json(orient='table'), len(
            countries), asset_list, checkoptions


@app.callback([Output('no_views', 'max')],
              [Input('dump_no', 'children')])
def max_views(max):
    return [int(max)]


@cg.callback([Output('wts_table_container', 'style'),
              Output('rhos_table_container', 'style'),
              Output('vol_table_container', 'style'),
              Output('cov_table_container', 'style'),
              Output('wts_table', 'columns'),
              Output('wts_table', 'data'),
              Output('rhos_table', 'columns'),
              Output('rhos_table', 'data'),
              Output('vol_table', 'columns'),
              Output('vol_table', 'data'),
              Output('cov_table', 'columns'),
              Output('cov_table', 'data'),
              Output('view_params', 'style'),
              Output('dump_no', 'children')],
             [Input('proceed_to_views', 'n_clicks')],
             [State('display_table', 'value'),
              State('asset_list', 'value')])
def upd_visibility(n_clicks, value, asset_list):
    if isinstance(asset_list, list):
        asset_list = asset_list[0]
    asset_list = list(asset_list.split(' '))
    disp_wts = disp_rhos = disp_vol = 'block' if 0 in value else 'none'
    if 1 in value:
        disp_rhos = disp_vol = 'none'
    disp_cov = 'block' if 1 in value else 'none'
    col_wts = [{'id': 'asset_name_wts', 'name': 'Assets'}] + [{'id': 'wts', 'name': 'Prior_weights'}]
    data_wts = [dict(asset_name_wts=asset) for asset in asset_list]
    col_vol = [{'id': 'asset_name_wts', 'name': 'Assets'}] + [{'id': 'vol', 'name': 'Prior_volatility'}]
    data_vol = [dict(asset_name_wts=asset) for asset in asset_list]
    col_rhos = [{'id': 'rhos', 'name': 'Rhos'}] + [{'id': asset, 'name': asset} for asset in asset_list]
    data_rhos = [dict(rhos=asset) for asset in asset_list]
    col_cov = [{'id': 'cov', 'name': 'cov_mat'}] + [{'id': asset, 'name': asset} for asset in asset_list]
    data_cov = [dict(cov=asset) for asset in asset_list]
    if n_clicks and n_clicks > 0:
        return {'display': disp_wts}, {'display': disp_rhos}, {'display': disp_vol}, {'display': disp_cov}, \
               col_wts, data_wts, col_rhos, data_rhos, col_vol, data_vol, col_cov, data_cov, {'display': 'flex',
                                                                                              'justify-content': 'space-evenly',
                                                                                              'padding-top': '25px'}, \
               len(asset_list)


@app.callback([Output('update', 'style'),
               Output('view_table_container', 'style'),
               Output('pick_table_container', 'style'),
               Output('omega_table_container', 'style'),
               Output('view_table', 'columns'),
               Output('view_table', 'data'),
               Output('pick_table', 'columns'),
               Output('pick_table', 'data'),
               Output('omega_table', 'columns'),
               Output('omega_table', 'data')],
              [Input('update_views', 'n_clicks')],
              [State('no_views', 'value'),
               State('asset_list', 'value')])
def upd_view_tables(n_clicks, no_views, asset_list):
    if isinstance(asset_list, list):
        asset_list = asset_list[0]
    asset_list = list(asset_list.split(' '))
    col_views = [{'id': 'views_no', 'name': 'Views'}] + [{'id': 'views', 'name': 'views'}]
    data_views = [dict(views_no=k) for k in range(1, no_views + 1)]
    col_pick = [{'id': 'pick', 'name': 'pick_mat'}] + [{'id': asset, 'name': asset} for asset in asset_list]
    data_pick = [dict(pick=k) for k in range(1, no_views + 1)]
    col_omega = [{'id': 'omega', 'name': 'Omega'}] + [{'id': k, 'name': k} for k in range(no_views)]
    data_omega = [dict(omega=k) for k in range(no_views)]
    if n_clicks and n_clicks > 0:
        return {'display': 'block'}, {'display': 'block'}, {
            'display': 'block'}, {'display': 'block'}, col_views, data_views, col_pick, data_pick, col_omega, data_omega


@app.callback([Output('disp_bl_cov', 'children'),
               Output('disp_omega', 'children'),
               Output('bl_results_table', 'columns'),
               Output('bl_results_table', 'data')],
              [Input('update', 'n_clicks')],
              [State('wts_table', 'data'),
               State('cov_table', 'data'),
               State('rhos_table', 'data'),
               State('vol_table', 'data'),
               State('view_table', 'data'),
               State('pick_table', 'data'),
               State('omega_table', 'data'),
               State('asset_list', 'value'),
               State('display_table', 'value'),
               State('dump_vw_df', 'children'),
               State('dump_wt_df', 'children'),
               State('dump_wt_hc_df', 'children'),
               State('dump_sigma_hc_df', 'children'),
               State('tau', 'value'),
               State('delta', 'value')])
def update_values(n_clicks, wts, cov, rhos, vol, views, pick_mat, omega_mat, asset_list, value, ind_vw_2014,
                  ind_mkt_wts_2014, wts_hc, sigma_hc, tau, delta):
    if isinstance(asset_list, list):
        asset_list = asset_list[0]
    asset_list = list(asset_list.split(' '))
    if ind_vw_2014 is not None and ind_mkt_wts_2014 is not None:
        ind_vw_2014 = pd.read_json(ind_vw_2014, orient='table').to_period('M')
        ind_mkt_wts_2014 = pd.read_json(ind_mkt_wts_2014, orient='table').to_period('M').iloc[0].rename('wts_prior')
        wts_prior = ind_mkt_wts_2014
        # corr_results = erk.get_corr_mat(ind_vw_2014, window=36)
        # corr_mat_group = corr_results[0]
        rhos_prior = ind_vw_2014.corr()
        vol_prior = erk.get_ann_vol(ind_vw_2014)
        sigma_prior = vol_prior @ vol_prior.T * rhos_prior
    elif wts_hc is not None and sigma_hc is not None:
        wts_prior = pd.read_json(wts_hc, orient='table').squeeze()
        sigma_prior = pd.read_json(sigma_hc, orient='table')
    else:
        wts_prior = pd.DataFrame(wts).set_index('asset_name_wts').squeeze().rename('wts_prior').fillna(0)
        wts_prior = wts_prior.astype('float')
        rhos_prior = 0
        vol_prior = 0
        if 1 not in value:
            rhos_prior = pd.DataFrame(rhos, columns=[asset for asset in asset_list]).astype('float')
            vol_prior = pd.DataFrame(vol)
            vol_prior = vol_prior.astype('float').fillna(0)
            sigma_prior = vol_prior @ vol_prior.T * rhos_prior
        else:
            sigma_prior = pd.DataFrame(cov, columns=[asset for asset in asset_list]).astype('float').fillna(0)
    views = pd.DataFrame(views)['views']
    views = views.astype('float').fillna(0)
    pick_df = pd.DataFrame(pick_mat, columns=[asset for asset in asset_list], index=views.index).astype('float').fillna(
        0)
    omega_df = pd.DataFrame(omega_mat).astype('float').fillna(0)
    omega_df.drop(axis=1, columns=['omega'], inplace=True)
    pi, bl_mu, bl_sigma, wts, omega_df = get_bl_results(wts_prior, sigma_prior, delta, tau, pick_df, views,
                                                        omega_df, scale=False, wts_he=True)
    bl_results_df = pd.DataFrame({'Asset': asset_list,
                                  'Cur_wts': wts_prior,
                                  'Equil_wts': wts[0],
                                  'Wts_Optimal': wts[1],
                                  'Wts_Diff': wts[2],
                                  'pi': pi,
                                  'bl_mu': bl_mu})
    float_cols = ['Cur_wts', 'Equil_wts', 'Wts_Optimal', 'Wts_Diff', 'pi', 'bl_mu']
    for col in float_cols:
        bl_results_df[col] = percent_format(bl_results_df[col])
    disp_bl_cov = 'Posterior cov: \n{}'.format(bl_sigma)
    disp_omega = 'Omega Matrix: \n{}'.format(omega_df)
    bl_results_columns = [{'name': col, 'id': col} for col in bl_results_df.columns]
    bl_results_data = bl_results_df.to_dict('records')
    if n_clicks and n_clicks > 0:
        return disp_bl_cov, disp_omega, bl_results_columns, bl_results_data


cg.register(app)
app.run_server()



Dash app running on http://127.0.0.1:8050/
